In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

seed=3
np.random.seed(seed)
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
labels_df = pd.read_csv('product_attributes_updated.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')

In [ ]:
labels_df = labels_df.drop(labels_df[labels_df.Style1=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style3=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style1=='French Country'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style2=='French Country'].index)

In [ ]:
for ColumnName in ['Style1','Style2','Style3']:
    labels_df[ColumnName].value_counts().plot(kind="bar")
    plt.show()

In [ ]:
one_hot1=pd.get_dummies(labels_df['Style1'])
one_hot2=pd.get_dummies(labels_df['Style2'])
one_hot3=pd.get_dummies(labels_df['Style3'])
Style_labels=one_hot1+one_hot2+one_hot3
Style_labels['Id']=labels_df['ProductID']
Style_labels.isnull().sum()

In [ ]:
 def image2np_array( filename ) :
    img = Image.open( filename )
    img.load()
    data = np.asarray( img, dtype='uint8' )
    return data
x_train_list=[]
y_train_list=[]

Style_labels['sum'] = Style_labels.sum(axis=1)


for row in Style_labels.itertuples():
    if (row[27]!=0):#26=Id  27=sum
        try:
            img = image2np_array('..\\ml_150x150\\'+row[26]+'.jpg').reshape(150,150,3)#26=Id  27=sum
        except (IOError, ValueError) as e:
            print('{0}'.format(e))
            continue
        x_train_list.append(img)
        y_train_list.append(Style_labels.loc[row[0],:'Vintage / Retro / Industrial'].values)

In [ ]:
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
x_train.shape

In [ ]:
x_validation=x_train[25000:]
y_validation=y_train[25000:]
x_train=x_train[:25000]
y_train=y_train[:25000]

In [ ]:
epochs = 30
batch_size = 32

input_shape = (150, 150, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dropout(0.5,seed=seed))
model.add(Dense(25))
model.add(Activation('sigmoid'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validation,y_validation),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs)


In [ ]:
model.save('style_classifier.h5')